In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt

In [2]:

# Charger le dataset
data = pd.read_csv("data_test2.csv")

# Suppression des colonnes non nécessaires
data = data.drop(['Libellé du département'], axis=1)
#data= data[['Année', 'Libellé du canton', 'Tour','nb_votants']]
data['True'] = data.groupby(['Année', 'Libellé du canton', 'Tour'])['nb_votants'].transform(lambda x:x==x.max())

#Convertir boolean
data['True'] = pd.to_numeric(data['True'])
data['Tour'] = pd.to_numeric(data['Tour'])
data['Année'] = pd.to_numeric(data['Année'])


#print(categorique)


# Conversion des colonnes en nombres
numeric_column = 'Estimations de population - Part des 0-24 ans - Isère'
#pp = 'Parti Politique'

#plt.figure(figsize=(6,6))
#plt.hist(data['Chômage Isere'])

# Séparation des données en features (X) et target (y)
data=data[['Chômage Isere','Libellé du canton','Parti Politique', numeric_column, 'Tour', 'Année','True']]
categorique= pd.get_dummies(data, columns=['Libellé du canton'], prefix='Libellé du canton')
categorique_pp= pd.get_dummies(categorique, columns=['Parti Politique'], prefix='Parti Politique')
#print(categorique_pp.columns)
#categorique_pp = categorique_pp.drop(['Libellé du canton','Parti Politique'], axis=1)
data_train = categorique_pp.loc[(categorique_pp['Année']!=2022) & (categorique_pp['Année']!=2017) ]
data_test = categorique_pp.loc[(categorique_pp['Année']==2022) | (categorique_pp['Année']==2017)]
#print(data_test.columns)
y_train = data_train['True']
y_test = data_test['True']
X_train = data_train.drop(['True'], axis=1)
X_test = data_test.drop(['True'], axis=1)
print(X_train.columns)





#X = data[['Chômage Isere', numeric_column, 'Tour', 'Année', 'Libellé du canton']]


# Divisez les données en ensembles d'entraînement et de test
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Liste des classificateurs à tester
classifiers = {
    'Random Forest': RandomForestClassifier(class_weight='balanced'),
    'SVM': SVC(class_weight='balanced'),
    'KNN': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced')
}

# Entraînement et évaluation des classificateurs
for name, clf in classifiers.items():
    clf.fit(X_train_scaled, y_train)
    predictions = clf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=1)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=1)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=1)
    
    print(f"Metrics for {name}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print("\nClassification Report:")
    print(classification_report(y_test, predictions, zero_division=1))
    print("-" * 50)

# Sélection du modèle le plus performant
best_classifier_name = max(classifiers, key=lambda k: precision_score(y_test, classifiers[k].predict(X_test_scaled), average='weighted', zero_division=1))
best_classifier = classifiers[best_classifier_name]

print(f"\nLe meilleur modèle est {best_classifier_name} avec une précision de {precision_score(y_test, best_classifier.predict(X_test_scaled), average='weighted', zero_division=1)}")


Index(['Chômage Isere',
       'Estimations de population - Part des 0-24 ans - Isère', 'Tour',
       'Année', 'Libellé du canton_Agnin', 'Libellé du canton_Allemond',
       'Libellé du canton_Allevard', 'Libellé du canton_Ambel',
       'Libellé du canton_Anjou', 'Libellé du canton_Annoisin-Chatelans',
       ...
       'Parti Politique_NPA', 'Parti Politique_PCF', 'Parti Politique_PRG',
       'Parti Politique_PS', 'Parti Politique_PT', 'Parti Politique_REC',
       'Parti Politique_RES', 'Parti Politique_RPR', 'Parti Politique_SP',
       'Parti Politique_UMP'],
      dtype='object', length=572)
Metrics for Random Forest:
Accuracy: 0.8780545112781954
Precision: 0.8158339890698906
Recall: 0.8780545112781954
F1 Score: 0.8218636735590285

Classification Report:
              precision    recall  f1-score   support

       False       0.88      1.00      0.94     11214
        True       0.36      0.00      0.01      1554

    accuracy                           0.88     12768
   macro